In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import sys

# Konstanten
RANDOM_SEED = 42
BATCH_SIZE = 32
EPOCHS = 30
LEARNING_RATE = 0.001
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MODEL_PATH = '/home/geiger/asl_detection/machine_learning/models/asl_now/best_model.pth'  # Pfad zum bestehenden Modell

# Dataset-Klasse
class HandSignDataset(Dataset):
    def __init__(self, keypoints, labels):
        self.keypoints = torch.FloatTensor(keypoints)
        self.labels = torch.LongTensor(labels)
    
    def __len__(self):
        return len(self.keypoints)
    
    def __getitem__(self, idx):
        return self.keypoints[idx], self.labels[idx]

# Modell-Definition
class HandSignNet(nn.Module):
    def __init__(self, num_classes=24):
        super(HandSignNet, self).__init__()
        
        # Feature Extraction Blocks
        self.features = nn.Sequential(
            nn.Linear(63, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3),
            
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3)
        )
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(128, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc

def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc, all_preds, all_labels

def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(15, 12))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig('confusion_matrix.png')
    plt.close()

def plot_training_history(train_losses, val_losses, train_accs, val_accs):
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train')
    plt.plot(val_losses, label='Validation')
    plt.title('Loss over epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Train')
    plt.plot(val_accs, label='Validation')
    plt.title('Accuracy over epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

def main(load_model=True):
    # Setze Seeds für Reproduzierbarkeit
    torch.manual_seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)
    
    # Alphabet-Definition
    alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y']
    
    # Lade Daten
    print("Lade Daten...")
    data = np.load('/home/geiger/asl_detection/machine_learning/datasets/asl_now/Keypoints_3/asl_keypoints.npz')
    keypoints = data['keypoints']
    labels = data['labels']
    
    # Split Daten
    X_train, X_val, y_train, y_val = train_test_split(
        keypoints, labels, test_size=0.2, random_state=RANDOM_SEED, stratify=labels
    )
    
    # Erstelle DataLoader
    train_dataset = HandSignDataset(X_train, y_train)
    val_dataset = HandSignDataset(X_val, y_val)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
    
    # Initialisiere Modell und lade vortrainiertes Modell, falls gewünscht
    print(f"Initialisiere Modell auf {DEVICE}...")
    model = HandSignNet().to(DEVICE)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
    
    initial_val_acc = 0
    if load_model and os.path.exists(MODEL_PATH):
        print(f"Lade vortrainiertes Modell von {MODEL_PATH}...")
        model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
        print("Vortrainiertes Modell erfolgreich geladen!")
        
        # Führe eine Validierung mit dem geladenen Modell durch
        print("Validiere geladenes Modell...")
        initial_val_loss, initial_val_acc, _, _ = validate(model, val_loader, criterion, DEVICE)
        print(f"Initiale Validierungs-Genauigkeit: {initial_val_acc:.2f}%")
    else:
        if load_model:
            print(f"Kein vortrainiertes Modell gefunden unter {MODEL_PATH}. Starte mit neuem Modell.")
        else:
            print("Training mit neuem Modell gestartet.")
    
    best_val_acc = initial_val_acc
    
    # Training
    print("Starte Training...")
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    
    for epoch in range(EPOCHS):
        # Training
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, DEVICE)
        
        # Validation
        val_loss, val_acc, val_preds, val_labels = validate(model, val_loader, criterion, DEVICE)
        
        # Learning Rate Anpassung
        scheduler.step(val_loss)
        
        # Speichere Metriken
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        # Speichere bestes Modell
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), MODEL_PATH)
            print(f"Neues bestes Modell gespeichert mit Accuracy: {val_acc:.2f}%")
        
        # Ausgabe
        print(f'Epoch {epoch+1}/{EPOCHS}:')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
        print('-' * 50)
    
    # Lade bestes Modell für finale Evaluation, aber nur wenn es existiert
    if os.path.exists(MODEL_PATH):
        print("Lade bestes Modell für finale Evaluation...")
        model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
        _, final_acc, final_preds, final_labels = validate(model, val_loader, criterion, DEVICE)
        
        # Plotte Ergebnisse
        plot_training_history(train_losses, val_losses, train_accs, val_accs)
        plot_confusion_matrix(final_labels, final_preds, alphabet)
    else:
        print("Kein gespeichertes Modell gefunden. Überspringe finale Evaluation.")
        # Verwende die Ergebnisse der letzten Epoche für die Plots
        final_preds = val_preds
        final_labels = val_labels
        plot_training_history(train_losses, val_losses, train_accs, val_accs)
        if len(final_labels) > 0:  # Nur wenn wir Validierungsdaten haben
            plot_confusion_matrix(final_labels, final_preds, alphabet)
    
    print(f"\nBeste Validierungs-Accuracy: {best_val_acc:.2f}%")
    if best_val_acc > initial_val_acc:
        print(f"Verbesserung gegenüber initialem Modell: +{best_val_acc - initial_val_acc:.2f}%")

if __name__ == "__main__":
    # Prüfe, ob ein Kommandozeilenargument übergeben wurde
    load_model = False  # Standard: Lade vorhandenes Modell
    
    main(load_model) 

Lade Daten...
Initialisiere Modell auf cpu...
Training mit neuem Modell gestartet.
Starte Training...
Neues bestes Modell gespeichert mit Accuracy: 99.00%
Epoch 1/30:
Train Loss: 0.4618, Train Acc: 90.86%
Val Loss: 0.0833, Val Acc: 99.00%
--------------------------------------------------
Epoch 2/30:
Train Loss: 0.0730, Train Acc: 98.47%
Val Loss: 0.4325, Val Acc: 84.86%
--------------------------------------------------
Epoch 3/30:
Train Loss: 0.0610, Train Acc: 98.37%
Val Loss: 0.2643, Val Acc: 91.96%
--------------------------------------------------
Epoch 4/30:
Train Loss: 0.0455, Train Acc: 98.75%
Val Loss: 0.9039, Val Acc: 76.33%
--------------------------------------------------
Neues bestes Modell gespeichert mit Accuracy: 99.73%
Epoch 5/30:
Train Loss: 0.0524, Train Acc: 98.48%
Val Loss: 0.0203, Val Acc: 99.73%
--------------------------------------------------
Epoch 6/30:
Train Loss: 0.0433, Train Acc: 98.76%
Val Loss: 0.0485, Val Acc: 98.17%
---------------------------------